# Exercício

Nesta aula, codificaremos junto com alguns dados e testaremos 3 métodos de árvore diferentes:

* Uma única árvore de decisão
* Uma floresta aleatória
* Um classificador de árvore gradiente impulsionado
    
Estaremos usando um conjunto de dados universitários para tentar classificar as faculdades como privadas ou públicas com base nestes recursos:

    Private A factor with levels No and Yes indicating private or public university
    Apps Number of applications received
    Accept Number of applications accepted
    Enroll Number of new students enrolled
    Top10perc Pct. new students from top 10% of H.S. class
    Top25perc Pct. new students from top 25% of H.S. class
    F.Undergrad Number of fulltime undergraduates
    P.Undergrad Number of parttime undergraduates
    Outstate Out-of-state tuition
    Room.Board Room and board costs
    Books Estimated book costs
    Personal Estimated personal spending
    PhD Pct. of faculty with Ph.D.’s
    Terminal Pct. of faculty with terminal degree
    S.F.Ratio Student/faculty ratio
    perc.alumni Pct. alumni who donate
    Expend Instructional expenditure per student
    Grad.Rate Graduation rate

In [1]:
#Tree methods Exemplo
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('treecode').getOrCreate()

In [2]:
# Carregar dados de treinamento
data = spark.read.csv('College.csv',inferSchema=True,header=True)

In [3]:
data.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



In [4]:
data.head()

Row(School='Abilene Christian University', Private='Yes', Apps=1660, Accept=1232, Enroll=721, Top10perc=23, Top25perc=52, F_Undergrad=2885, P_Undergrad=537, Outstate=7440, Room_Board=3300, Books=450, Personal=2200, PhD=70, Terminal=78, S_F_Ratio=18.1, perc_alumni=12, Expend=7041, Grad_Rate=60)

### Formatação de dados do Spark

In [5]:
# Algumas coisas que precisamos fazer antes que o Spark possa aceitar os dados!
# Precisa ter a forma de duas colunas
# ("label","features")

# Importe VectorAssembler e Vectors
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [6]:
data.columns

['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [7]:
assembler = VectorAssembler(
  inputCols=['Apps',
             'Accept',
             'Enroll',
             'Top10perc',
             'Top25perc',
             'F_Undergrad',
             'P_Undergrad',
             'Outstate',
             'Room_Board',
             'Books',
             'Personal',
             'PhD',
             'Terminal',
             'S_F_Ratio',
             'perc_alumni',
             'Expend',
             'Grad_Rate'],
              outputCol="features")

In [8]:
output = assembler.transform(data)

Lide com a coluna privada sendo "sim" ou "não"

In [9]:
from pyspark.ml.feature import StringIndexer

In [10]:
indexer = StringIndexer(inputCol="Private", outputCol="PrivateIndex")
output_fixed = indexer.fit(output).transform(output)

In [11]:
final_data = output_fixed.select("features",'PrivateIndex')

In [12]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

### Os classificadores


In [13]:
from pyspark.ml.classification import DecisionTreeClassifier,GBTClassifier,RandomForestClassifier
from pyspark.ml import Pipeline

Crie todos os três modelos:

In [14]:
# Use principalmente os padrões para tornar esta comparação "justa"
dtc = DecisionTreeClassifier(labelCol='PrivateIndex',featuresCol='features')
rfc = RandomForestClassifier(labelCol='PrivateIndex',featuresCol='features')
gbt = GBTClassifier(labelCol='PrivateIndex',featuresCol='features')

Treine todos os três modelos:

In [15]:
# Treine os modelos (seus três modelos, por isso pode levar algum tempo)
dtc_model = dtc.fit(train_data)
rfc_model = rfc.fit(train_data)
gbt_model = gbt.fit(train_data)

## Comparação de Modelos

Vamos comparar cada um desses modelos!

In [16]:
dtc_predictions = dtc_model.transform(test_data)
rfc_predictions = rfc_model.transform(test_data)
gbt_predictions = gbt_model.transform(test_data)

** Métricas de avaliação: **

In [17]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [18]:
# Selecione (previsão, rótulo verdadeiro) e calcule o erro de teste
acc_evaluator = MulticlassClassificationEvaluator(labelCol="PrivateIndex", predictionCol="prediction", metricName="accuracy")

In [19]:
dtc_acc = acc_evaluator.evaluate(dtc_predictions)
rfc_acc = acc_evaluator.evaluate(rfc_predictions)
gbt_acc = acc_evaluator.evaluate(gbt_predictions)

In [21]:
print("Aqui estão os resultados!")
print('-'*80)
print('Uma única árvore de decisão teve uma precisão de: {0:2.2f}%'.format(dtc_acc*100))
print('-'*80)
print('Um conjunto de floresta aleatório teve uma precisão de: {0:2.2f}%'.format(rfc_acc*100))
print('-'*80)
print('Um conjunto usando GBT teve uma precisão de: {0:2.2f}%'.format(gbt_acc*100))

Aqui estão os resultados!
--------------------------------------------------------------------------------
Uma única árvore de decisão teve uma precisão de: 92.18%
--------------------------------------------------------------------------------
Um conjunto de floresta aleatório teve uma precisão de: 95.06%
--------------------------------------------------------------------------------
Um conjunto usando GBT teve uma precisão de: 92.59%


Interessante! Atribuição opcional - brinque com os parâmetros de cada um desses modelos, você consegue obter um pouco mais de precisão deles? Ou os dados são o fator limitante?